# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv 


In [ ]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [ ]:
import os
from glob import glob

# Write your code below. 
#LOAD
PRICE_DATA = os.getenv("PRICE_DATA")

parquet_files = glob(
    os.path.join(PRICE_DATA, "**/*.parquet"),
    recursive=True
)

parquet_files[:5]

2010


['../../05_src/data/prices/BKTI/BKTI_2012/part.0.parquet',
 '../../05_src/data/prices/BKTI/BKTI_2012/part.1.parquet',
 '../../05_src/data/prices/BKTI/BKTI_2015/part.0.parquet',
 '../../05_src/data/prices/BKTI/BKTI_2015/part.1.parquet',
 '../../05_src/data/prices/BKTI/BKTI_2014/part.0.parquet']

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [9]:
# Write your code below.
#TRANSFORM

import dask.dataframe as dd # already done

# Read parquet files into Dask dataframe 
dd_px = dd.read_parquet(parquet_files).set_index("ticker") 
dd_px

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year
npartitions=60,,,,,,,,,
A,datetime64[ns],float64,float64,float64,float64,float64,float64,string,int32
ACB,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
ZEUS,...,...,...,...,...,...,...,...,...
ZEUS,...,...,...,...,...,...,...,...,...


In [11]:
dd_shift = (
    dd_px
        .groupby('ticker', group_keys=False) # Group the rows of dataframe by Ticker
        .apply(
            lambda x: x.sort_values('Date', ascending = True)
                        # Assign lags 
                       .assign(
                           Close_lag_1 = x['Close'].shift(1),
                           Adj_Close_lag_1 = x['Adj Close'].shift(1)
                       ),
            # Include meta -> where to put the new data
            meta=dd_px._meta.assign(
                Close_lag_1='f8',
                Adj_Close_lag_1='f8'
            )

        ))

dd_feat = dd_shift.assign(
    returns = lambda x: x['Close'] / x['Close_lag_1'] - 1,
    hi_lo_range = lambda x: x['High'] - x['Low']
)

dd_feat

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range
npartitions=60,,,,,,,,,,,,,
A,datetime64[ns],float64,float64,float64,float64,float64,float64,string,int32,object,object,object,float64
ACB,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEUS,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEUS,...,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [14]:
# Write your code below.
# Import pandas
import pandas as pd

# Convert to a pandas df
pd_dd_feat = dd_feat.compute()


In [ ]:
pd_dd_feat = pd_dd_feat.sort_values(['ticker','Date'])
pd_dd_feat['rolling_avg'] = pd_dd_feat.groupby('ticker')['returns'].rolling(10).mean().reset_index(level=0, drop=True)


In [17]:
pd_dd_feat


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,rolling_avg
ticker,,,,,,,,,,,,,,
A,1999-11-18,32.546494,35.765381,28.612303,31.473534,27.068665,62546300.0,A.csv,1999,NaN,NaN,NaN,7.153078,NaN
A,1999-11-19,30.713520,30.758226,28.478184,28.880543,24.838577,15234100.0,A.csv,1999,39.360001,37.599323,-0.266246,2.280043,NaN
A,1999-11-22,29.551144,31.473534,28.657009,31.473534,27.068665,6577800.0,A.csv,1999,39.790001,38.010090,-0.209009,2.816525,NaN
A,1999-11-23,30.400572,31.205294,28.612303,28.612303,24.607880,5975600.0,A.csv,1999,38.750000,37.016602,-0.261618,2.592991,NaN
A,1999-11-24,28.701717,29.998211,28.612303,29.372318,25.261524,4843200.0,A.csv,1999,38.919998,37.179005,-0.245316,1.385908,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEUS,2020-03-26,9.610000,9.940000,9.260000,9.590000,9.590000,60500.0,ZEUS.csv,2020,6.870000,6.292048,0.395924,0.679999,0.525542
ZEUS,2020-03-27,9.330000,9.330000,8.700000,8.700000,8.700000,52900.0,ZEUS.csv,2020,6.990000,6.401954,0.244635,0.630000,0.496311
ZEUS,2020-03-30,8.810000,9.760000,8.700000,9.680000,9.680000,73700.0,ZEUS.csv,2020,7.630000,6.988110,0.268676,1.060000,0.465918


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
    # No it wasnt, you could use Dask using groupby and assign like we did previously
+ Would it have been better to do it in Dask? Why?
    # Probably since using pandas it doesnt ensure the rolling window is per ticker

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.